In [1]:
import importlib
import torch
import torchvision


from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from pathlib import Path

import time
import learning_loop

from learning_loop import BATCH_SIZE

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [3]:
import models

BarlowTwins = models.BarlowTwins

In [4]:
def reload():
    importlib.reload(learning_loop)
    importlib.reload(models)

    global BATCH_SIZE, BarlowTwins, l_loop

    l_loop = learning_loop.learning_loop
    BATCH_SIZE = learning_loop.BATCH_SIZE
    BarlowTwins = models.BarlowTwins

In [5]:
stl = datasets.STL10(root="data", split="train", download=True, transform=ToTensor())

Files already downloaded and verified


In [6]:
stl_loader = DataLoader(stl, batch_size=BATCH_SIZE)
data, cls = next(iter(stl_loader))

In [7]:
reload()
model = BarlowTwins(BATCH_SIZE).to("cuda")

In [8]:
eigen_values = []

def add_eigen_values(eigen):
    eigen_values.append(eigen)

In [9]:
reload()
l_loop(model, stl, add_eigen_values)

/home/ubuntu/anaconda3/envs/pytorch_latest_p37/lib/python3.7/site-packages/ipykernel/__main__.py -f /home/ubuntu/.local/share/jupyter/runtime/kernel-4ed6c8de-8e73-4049-a502-c7c84ac5dd5c.json
loss tensor(454.7486, device='cuda:0', grad_fn=<AddBackward0>)


RuntimeError: eig_backward: Backward calculation does not support complex eigenvalues at the moment.

In [14]:
reps = model.forward_reps(data.to(device))

cov = (reps.T @ reps) / model.batch_size

vals, vecs = torch.eig(cov, eigenvectors=True)
d = torch.diag((vals[:, 0] ** .5))
d[d != d] = 0

In [22]:
vals[:, 0].abs().min()

tensor(3.2521e-10, device='cuda:0', grad_fn=<MinBackward1>)